In [1]:
!git clone https://github.com/SobhanRjz/DL_Detectron2.git
%cd DL_Detectron2

Cloning into 'DL_Detectron2'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 141 (delta 66), reused 107 (delta 38), pack-reused 0 (from 0)
Receiving objects: 100% (141/141), 4.43 MiB | 18.24 MiB/s, done.
Resolving deltas: 100% (66/66), done.
/content/DL_Detectron2


In [2]:
# prompt: get from drive folder

from google.colab import drive
drive.mount('/content/drive')
# Then you can use standard file operations to access the files.
# For example, to list the files in a folder:
!ls "/content/drive/My Drive/Colab Notebooks"


Mounted at /content/drive
 images.rar	   NeuralNetwork.ipynb			     Untitled0.ipynb
 model_final.pth  'Train_Detectron2_(7)_(9)_(1)_(6).ipynb'


In [3]:
# prompt: makedir if not exist /content/DL_Detectron2/DataSets/images
import os
# Create the directory if it does not exist
dir_path = "/content/DL_Detectron2/DataSets/images"
if not os.path.exists(dir_path):

    os.makedirs(dir_path)
    print(f"Directory '{dir_path}' created successfully.")
else:
    print(f"Directory '{dir_path}' already exists.")

Directory '/content/DL_Detectron2/DataSets/images' created successfully.


In [ ]:
# prompt: unzip rar file

!unrar x "/content/drive/My Drive/Colab Notebooks/images.rar" "/content/DL_Detectron2/DataSets/images"


Streaming output truncated to the last 5000 lines.
Extracting  /content/DL_Detectron2/DataSets/images/train/07A-3541-007A-3540-01_frame28497_png_jpg.rf.59801597302f589e5c1f6a0884a6e36f.jpg       3%  OK 
Extracting  /content/DL_Detectron2/DataSets/images/train/07A-3913-007A-3912-01_frame15799_png_jpg.rf.d7f736c29045d439b80ca727a97b57fc.jpg       3%  OK 
Extracting  /content/DL_Detectron2/DataSets/images/train/07A-3913-007A-3912-01_frame18228_png_jpg.rf.bc778df3a03611423ad098038a7f2463.jpg       3%  OK 
Extracting  /content/DL_Detectron2/DataSets/images/train/07A-3913-007A-3912-01_frame18746_png_jpg.rf.88b41adc978112a07d7dfa2121cde996.jpg       3%  OK 
Extracting  /content/DL_Detectron2/DataSets/images/train/07A-3913-007A-3912-01_frame18942_png_jpg.rf.e3253ec408b8e0973958351608e87256.jpg       3%  OK 
Extracting  /content/DL_Detectron2/DataSets/images/train/07A-3913-007A-3912-01_frame18984_png_jpg.rf.1ac2370ef11cc16971604d8aa1b31a63.jpg     

In [ ]:

# Look at training curves in tensorboard:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

from google.colab.patches import cv2_imshow



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 7.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15837, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 15837 (delta 30), reused 12 (delta 12), pack-reused 15771 (from 2)
Receiving objects: 100% (15837/15837), 6.40 MiB | 23.33 MiB/s, done.
Resolving deltas: 100% (11532/11532), done.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your e

In [ ]:
import logging
import sys

# Configure the logger
logging.basicConfig(
    level=logging.INFO,  # Set the logging level to INFO
    format="%(asctime)s [%(levelname)s] %(message)s",
    force=True, # Resets any previous configuration
    handlers=[
        logging.StreamHandler(sys.stdout)  # Stream logs to Colab's cell output
    ]
)

# Create logger instance
logger = logging.getLogger()
logger.info("Logger is configured!")


2025-01-23 16:50:53,836 [INFO] Logger is configured!


In [ ]:
import os
import sys
# Get the parent directory path
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)


import logging
from pathlib import Path
import torch
from Config.basic_config import BASE_PATH, DATA_PATH, OUTPUT_PATH, DEVICE
from Config.basic_config import detectron2_logger as logger
from Config.detectron2_config import DetectronConfig
from Config.dataset_config import DatasetConfig
from PreProcess.preprocess_COCO import COCOJsonProcessor
from Train import mainTrain
from detectron2.engine import launch


In [ ]:
# Use pathlib for more robust path handling
json_paths = [
    Path(DATA_PATH) / split / "_annotations.coco.json"
    for split in ["train", "test", "valid"]
]

logger.info("Processing COCO JSON files...")
# coco_processor = COCOJsonProcessor([str(p) for p in json_paths])
# coco_processor.process_files()

logger.info("Registering datasets...")
DatasetConfig().register_datasets()


2025-01-23 16:50:59,293 [INFO] Processing COCO JSON files...
2025-01-23 16:50:59,296 [INFO] Registering datasets...
['coco_2014_train', 'coco_2014_val', 'coco_2014_minival', 'coco_2014_valminusminival', 'coco_2017_train', 'coco_2017_val', 'coco_2017_test', 'coco_2017_test-dev', 'coco_2017_val_100', 'keypoints_coco_2014_train', 'keypoints_coco_2014_val', 'keypoints_coco_2014_minival', 'keypoints_coco_2014_valminusminival', 'keypoints_coco_2017_train', 'keypoints_coco_2017_val', 'keypoints_coco_2017_val_100', 'coco_2017_train_panoptic_separated', 'coco_2017_train_panoptic_stuffonly', 'coco_2017_train_panoptic', 'coco_2017_val_panoptic_separated', 'coco_2017_val_panoptic_stuffonly', 'coco_2017_val_panoptic', 'coco_2017_val_100_panoptic_separated', 'coco_2017_val_100_panoptic_stuffonly', 'coco_2017_val_100_panoptic', 'lvis_v1_train', 'lvis_v1_val', 'lvis_v1_test_dev', 'lvis_v1_test_challenge', 'lvis_v0.5_train', 'lvis_v0.5_val', 'lvis_v0.5_val_rand_100', 'lvis_v0.5_test', 'lvis_v0.5_train_

In [ ]:
from collections import Counter
from detectron2.data import DatasetCatalog, MetadataCatalog
import matplotlib.pyplot as plt
import matplotlib

# Load dataset

dataset_dicts = DatasetCatalog.get("my_dataset_train")
metadata = MetadataCatalog.get("my_dataset_train")



2025-01-23 16:51:00,691 [INFO] Loading /content/DL_Detectron2/DataSets/images/train/_annotations.coco.json takes 1.39 seconds.
2025-01-23 16:51:00,699 [INFO] Loaded 3362 images in COCO format from /content/DL_Detectron2/DataSets/images/train/_annotations.coco.json


In [ ]:
from Colab.detectron2_config_colab import DetectronConfigColab
def setup_training():
    """Configure training settings and get GPU count"""
    logger.info("Loading Detectron2 configuration...")


    detectron_config = DetectronConfigColab()
    cfg = detectron_config.get_cfg()

    num_gpus = torch.cuda.device_count()
    logger.info(f"Number of GPUs available: {num_gpus}")

    return cfg, num_gpus
# Setup training configuration
cfg, num_gpus = setup_training()

2025-01-23 16:51:03,699 [INFO] Loading Detectron2 configuration...
2025-01-23 16:51:05,150 [INFO] Loading /content/DL_Detectron2/DataSets/images/train/_annotations.coco.json takes 1.45 seconds.
2025-01-23 16:51:05,160 [INFO] Loaded 3362 images in COCO format from /content/DL_Detectron2/DataSets/images/train/_annotations.coco.json
2025-01-23 16:51:07,725 [INFO] NumExpr defaulting to 2 threads.
[[ 72.46802238  81.30803832]
 [102.67577809 215.83000612]
 [407.67798014 238.05904316]
 [158.58670886 429.44201665]
 [208.21064729 115.79574261]]
Calculated Anchor Ratios: [0.49280301 2.25208321 3.79903093 1.22957021 6.34545292]


Calculating mean and std:   0%|          | 0/53 [00:01<?, ?it/s]

Total Iterations: 32.83203125
Configuration saved to /content/DL_Detectron2/output/mask_rcnn_X_101_32x8d_FPN_3x.yaml
2025-01-23 16:51:10,077 [INFO] Number of GPUs available: 1


In [ ]:
logger.info("Launching training...")
IsResume = True
if IsResume:
import shutil
import os

# Source and destination paths
source_path = "/content/drive/My Drive/Colab Notebooks/model_final.pth"
dest_path = os.path.join(OUTPUT_PATH, "model_final.pth")

    
mainTrain(cfg, "custom", True)


2025-01-23 16:51:12,485 [INFO] Launching training...
2025-01-23 16:51:12,486 [INFO] Initializing custom trainer...
2025-01-23 16:51:14,399 [INFO] Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3

X-101-32x8d.pkl: 356MB [00:01, 196MB/s]                           

2025-01-23 16:51:19,146 [INFO] Successfully downloaded /root/.torch/iopath_cache/detectron2/ImageNetPretrained/FAIR/X-101-32x8d.pkl. 356008631 bytes.
2025-01-23 16:51:19,149 [INFO] URL https://dl.fbaipublicfiles.com/detectron2/ImageNetPretrained/FAIR/X-101-32x8d.pkl cached in /root/.torch/iopath_cache/detectron2/ImageNetPretrained/FAIR/X-101-32x8d.pkl
2025-01-23 16:51:19,151 [INFO] [Checkpointer] Loading from /root/.torch/iopath_cache/detectron2/ImageNetPretrained/FAIR/X-101-32x8d.pkl ...


2025-01-23 16:51:19,408 [INFO] Renaming Caffe2 weights ......
2025-01-23 16:51:19,720 [INFO] Following weights matched with submodule backbone.bottom_up - Total num: 105
2025-01-23 16:51:19,904 [WARNING] Some model parameters or buffers are not found in the checkpoint:
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.deconv.{bias, weight}
roi_heads.mask_head.mask_fcn1.{bias, weight}
roi_heads.m

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


2025-01-23 16:51:23,691 [INFO] Loading /content/DL_Detectron2/DataSets/images/train/_annotations.coco.json takes 1.18 seconds.
2025-01-23 16:51:24,111 [INFO] Loaded 3362 images in COCO format from /content/DL_Detectron2/DataSets/images/train/_annotations.coco.json
2025-01-23 16:51:24,152 [INFO] Cat ID 0: freq=0.43, rep=1.00
2025-01-23 16:51:24,153 [INFO] Cat ID 1: freq=0.24, rep=1.00
2025-01-23 16:51:24,156 [INFO] Cat ID 2: freq=0.19, rep=1.00
2025-01-23 16:51:24,158 [INFO] Cat ID 3: freq=0.08, rep=1.12
2025-01-23 16:51:24,160 [INFO] Cat ID 4: freq=0.12, rep=1.00
2025-01-23 16:51:24,161 [INFO] Cat ID 5: freq=0.27, rep=1.00
2025-01-23 16:51:24,163 [INFO] Cat ID 6: freq=0.21, rep=1.00
2025-01-23 16:51:24,164 [INFO] Cat ID 7: freq=0.09, rep=1.07
2025-01-23 16:51:24,171 [INFO] Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
2025-01-23 16:51:24,176 [INFO] Serializing 3362 elements to byte tensors and concatenating them all ...
2025-01-23 16:51:24,306 [IN

/usr/local/lib/python3.11/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


2025-01-23 16:51:32,693 [INFO] New best loss: 3.8331
2025-01-23 16:51:39,272 [INFO] New best loss: 3.7526
2025-01-23 16:51:57,884 [INFO] New best loss: 3.6423
2025-01-23 16:52:12,769 [INFO] New best loss: 3.4821
2025-01-23 16:52:16,579 [INFO] New best loss: 3.4054
2025-01-23 16:52:24,136 [INFO] New best loss: 3.2784
2025-01-23 16:52:31,641 [INFO] New best loss: 3.1752
2025-01-23 16:52:39,281 [INFO] New best loss: 3.0034
2025-01-23 16:52:43,013 [INFO]  iter: 19  total_loss: 3.645  loss_cls: 2.083  loss_box_reg: 0.03418  loss_mask: 0.6928  loss_rpn_cls: 0.6905  loss_rpn_loc: 0.1677     lr: 1.9981e-05  max_mem: 9645M
2025-01-23 16:52:43,022 [INFO] New best loss: 2.9023
2025-01-23 16:52:46,859 [INFO] New best loss: 2.8272


In [ ]:
# Enable synchronous CUDA execution for debugging
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Check CUDA availability
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'Using GPU: {torch.cuda.get_device_name(0)}')
else:
    raise RuntimeError('CUDA is not available')


Using GPU: Tesla T4


In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_max_memory_cached()

/usr/local/lib/python3.11/dist-packages/torch/cuda/memory.py:365: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/cuda/memory.py:391: FutureWarning: torch.cuda.reset_max_memory_cached now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
